In [1]:
import pandas as pd
import numpy as np

In [2]:
def previsao_consumo_diario(data_inicio, 
                            data_fim, 
                            dias_rampa, 
                            valor_arr, 
                            cliente, 
                            oportunidade = '',
                            tipo_forecast = '', 
                            pct_inicial_rampa=0.01, 
                            pct_final_rampa=0.90):
    idx = pd.date_range(data_inicio, data_fim)
    ramp_linspace = np.linspace(start=pct_inicial_rampa, stop=pct_final_rampa, num=dias_rampa)
    
    if len(idx) - dias_rampa >= 0:
        ramp_array = np.append(ramp_linspace, np.ones(len(idx) - dias_rampa))
    else:
        ramp_array = ramp_linspace[0: len(idx)]
    
    valor_diario = valor_arr / 365
    previsao_consumo_array = valor_diario * ramp_array
    
    df = pd.DataFrame(
        {
            'Consumo Projetado USD': previsao_consumo_array,
            'Cliente': cliente,
            'Oportunidade': oportunidade,
            'Tipo Forecast': tipo_forecast
        },
        index=idx
    )
    df.index.name = 'Data'
    
    return df

In [3]:
oportunidades = pd.read_excel('Lista de Oportunidades.xlsx').to_dict(orient='records')

In [4]:
lista_previsoes = [previsao_consumo_diario(**op) for op in oportunidades]

In [5]:
previsoes_df = pd.concat(lista_previsoes)

In [6]:
previsoes_df

,Consumo Projetado USD,Cliente,Oportunidade,Tipo Forecast
Data,,,,
2021-06-01,246.819178,BBTS,BBTS ExaCC,Forecast
2021-06-02,246.819178,BBTS,BBTS ExaCC,Forecast
2021-06-03,246.819178,BBTS,BBTS ExaCC,Forecast
2021-06-04,246.819178,BBTS,BBTS ExaCC,Forecast
2021-06-05,246.819178,BBTS,BBTS ExaCC,Forecast
...,...,...,...,...
2022-05-27,202.739726,PREVI,PREVI Move to Cloud,Pipeline
2022-05-28,205.479452,PREVI,PREVI Move to Cloud,Pipeline
2022-05-29,208.219178,PREVI,PREVI Move to Cloud,Pipeline


In [7]:
with pd.ExcelWriter('Previsão de Consumo.xlsx') as writer:
    previsoes_df.to_excel(writer, sheet_name='Previsão de Consumo')